In [ ]:
import os
import sys
import tracemalloc
tracemalloc.start()

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pyeCAP
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import signal

#from HF_functions import pentaPLOT, find_HF, get_parameters

In [ ]:
'Functions'
def pentaPLOT(param, ECAP_channel, params_to_tank_dict, metaDF, physDF, ECAP_data, EMG_data, TriCAP_data, phys_data, phys_channel, auto_scale_factor, remove_stim_artifact = False, offsets = None, save_fig = False):
    
    if isinstance(param, list):
        raise Exception("Param must be a single parameter, not a list.")
    
    tank_ID = params_to_tank_dict[param[0]]
    HF_duration = metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['HF Duration (s)'].item()
    
    """Grabs the pulse # of HF onset/offset using the standard deviation of the each pulse"""
    HF_onset_pulse, HF_offset_pulse = find_HF(param, ECAP_data, ECAP_channel, metaDF, tank_ID, plot=False)
    print("HF onset pulse #: " + str(HF_onset_pulse))
    print("HF offset pulse #: " + str(HF_offset_pulse)) 
    
    """Pull time values, units converted to milliseconds."""
    plot_time = np.squeeze(ECAP_data.time(param) * 1e3) 
    
    """Pull signals for pre, during, and post-HF pulse.  Units converted to microvolts"""   
    pre_HF_ECAP = np.squeeze(ECAP_data.mean(parameters=param, channels = ECAP_channel, bin=[0,HF_onset_pulse-1])[param]) * 1e6
    HF_ECAP = np.squeeze(ECAP_data.mean(parameters=param, channels = ECAP_channel, bin=[HF_onset_pulse + 1,HF_offset_pulse - 1])[param]) * 1e6
    post_HF_ECAP = np.squeeze(ECAP_data.mean(parameters=param, channels = ECAP_channel, bin=[HF_offset_pulse + 1, ECAP_data.stim.parameters.loc[param, 'pulse count']])[param]) * 1e6

    pre_HF_TriCAP = np.squeeze(TriCAP_data.mean(parameters=param, channels = 0, bin=[0,HF_onset_pulse-1])[param]) * 1e6
    HF_TriCAP = np.squeeze(TriCAP_data.mean(parameters=param, channels = 0, bin=[HF_onset_pulse + 1,HF_offset_pulse -1])[param]) * 1e6
    post_HF_TriCAP = np.squeeze(TriCAP_data.mean(parameters=param, channels = 0, bin=[HF_offset_pulse + 1, TriCAP_data.stim.parameters.loc[param, 'pulse count']])[param]) * 1e6

    pre_HF_EMG = np.squeeze(EMG_data.mean(parameters=param, bin=[0,HF_onset_pulse-1])[param]) * 1e6
    HF_EMG = np.squeeze(EMG_data.mean(parameters=param, bin=[HF_onset_pulse + 1,HF_offset_pulse -1])[param]) * 1e6
    post_HF_EMG = np.squeeze(EMG_data.mean(parameters=param, bin=[HF_offset_pulse + 1, EMG_data.stim.parameters.loc[param, 'pulse count']])[param]) * 1e6
    
    if offsets is not None:        
        if not isinstance(offsets, list):
            raise Exception("'offsets' must be a list of integer values of format [ECAP_offset, TriCAP_offset, EMG_offset] or 'None'.")
        else:                
            ECAP_dy = np.std(pre_HF_ECAP) * offsets[0]
            TriCAP_dy = np.std(pre_HF_TriCAP) * offsets[1]
            EMG_dy = np.std(pre_HF_EMG, axis = 1) * offsets[2]
            
            pre_HF_ECAP = pre_HF_ECAP + ECAP_dy
            post_HF_ECAP = post_HF_ECAP - ECAP_dy
            
            pre_HF_TriCAP = pre_HF_TriCAP + TriCAP_dy
            post_HF_TriCAP = post_HF_TriCAP - TriCAP_dy
            
            pre_HF_EMG[0] = pre_HF_EMG[0] + EMG_dy[0]
            post_HF_EMG[0] = post_HF_EMG[0] - EMG_dy[0]
            pre_HF_EMG[1] = pre_HF_EMG[1] + EMG_dy[1]
            post_HF_EMG[1] = post_HF_EMG[1] - EMG_dy[1]            
            pre_HF_EMG[2] = pre_HF_EMG[2] + EMG_dy[2]
            post_HF_EMG[2] = post_HF_EMG[2] - EMG_dy[2]
    
    fig,ax = plt.subplots(num =1, nrows = 3, ncols=2, figsize=(15,10), clear = True)
    ax = ax.ravel()

    """ ECAP """
    ax[0].plot(plot_time, pre_HF_ECAP, color = 'b', label = 'Pre-HF')
    ax[0].plot(plot_time, HF_ECAP, color = 'r', label = 'During HF')
    ax[0].plot(plot_time, post_HF_ECAP, color = 'g', label = 'Post-HF')

    if remove_stim_artifact:
        ECAP_std_data = np.std(pre_HF_ECAP[60:])
    else:
        ECAP_std_data = np.std(pre_HF_ECAP)
    
    if offsets is not None:
        ECAP_y_lim = [(-ECAP_std_data * auto_scale_factor) - ECAP_dy, (ECAP_std_data * auto_scale_factor) + ECAP_dy]    
    else:        
        ECAP_y_lim = [-ECAP_std_data * auto_scale_factor, ECAP_std_data * auto_scale_factor] 
    ax[0].set(ylim=(ECAP_y_lim[0],ECAP_y_lim[1]), xlim=(0,20), title='ECAP', ylabel = 'Voltage (uV)')
    ax[0].legend(loc='upper right')

    """Tripolar ECAP"""
    ax[1].plot(plot_time, pre_HF_TriCAP, color = 'b', label = 'Pre-HF')
    ax[1].plot(plot_time, HF_TriCAP, color = 'r', label = 'During HF')
    ax[1].plot(plot_time, post_HF_TriCAP, color = 'g', label = 'Post-HF')

    if remove_stim_artifact:
        triCAP_std_data = np.std(pre_HF_TriCAP[60:])
    else:
        triCAP_std_data = np.std(pre_HF_TriCAP)

    if offsets is not None:
        triCAP_y_lim = [(-triCAP_std_data * auto_scale_factor) - TriCAP_dy, (triCAP_std_data * auto_scale_factor) + TriCAP_dy]    
    else:
        triCAP_y_lim = [-triCAP_std_data * auto_scale_factor, triCAP_std_data * auto_scale_factor] 
    ax[1].set(ylim=(triCAP_y_lim[0],triCAP_y_lim[1]), xlim=(0,20), title='Tripolar ECAP', ylabel = 'Voltage (uV)')
    ax[1].legend(loc='upper right')

    #TODO: Add EMG channel ID's to metadata DF so that the function can autoplot the correct EMG
    """ CA EMG """
    CA_chan = metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['CA EMG Channel'].item()

    ax[2].plot(plot_time, pre_HF_EMG[CA_chan], color = 'b', label = 'Pre-HF')
    ax[2].plot(plot_time, HF_EMG[CA_chan], color = 'r', label = 'During HF')
    ax[2].plot(plot_time, post_HF_EMG[CA_chan], color = 'g', label = 'Post-HF')
    
    EMG1_std_data = np.std(pre_HF_EMG[CA_chan])
    if offsets is not None:
        EMG1_y_lim = [(-EMG1_std_data * auto_scale_factor) - EMG_dy[CA_chan], (EMG1_std_data * auto_scale_factor) + EMG_dy[CA_chan]]    
    else:
        EMG1_y_lim = [-EMG1_std_data * auto_scale_factor, EMG1_std_data * auto_scale_factor] 

    ax[2].legend(loc='upper right')
    ax[2].set(ylabel = 'Voltage (uV)', title='Cricoarytenoid EMG', xlim = (0,20), ylim=(EMG1_y_lim[0],EMG1_y_lim[1]))

    """CT EMG"""
    CT_chan = metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['CT EMG Channel'].item()

    ax[3].plot(plot_time, pre_HF_EMG[CT_chan], color = 'b', label = 'Pre-HF')
    ax[3].plot(plot_time, HF_EMG[CT_chan], color = 'r', label = 'During HF')
    ax[3].plot(plot_time, post_HF_EMG[CT_chan], color = 'g', label = 'Post-HF')

    EMG2_std_data = np.std(pre_HF_EMG[CT_chan])
    if offsets is not None:
        EMG2_y_lim = [(-EMG2_std_data * auto_scale_factor) - EMG_dy[CT_chan], (EMG2_std_data * auto_scale_factor) + EMG_dy[CT_chan]]    
    else:
        EMG2_y_lim = [-EMG2_std_data * auto_scale_factor, EMG2_std_data * auto_scale_factor] 

    ax[3].legend(loc='upper right')
    ax[3].set(xlabel = 'Time (ms)', title='Cricothyroid EMG', xlim = (0,20), ylim=(EMG2_y_lim[0],EMG2_y_lim[1]))

    """ Cardiac """
    physDF_idx = physDF.loc[ physDF['Stim Parameter'] == param].index
    HR_plot_start = physDF.loc[physDF_idx, 'Start Index'].item() - 30001
    HR_plot_stop = physDF.loc[physDF_idx, 'Stop Index'].item() + 30001
    HR_plot_time = np.arange(0, (HR_plot_stop - HR_plot_start) / 1000, .001)

    #For adding shading to plot
    pulse_start = 30001
    pulse_duration = physDF.loc[physDF_idx, 'Stop Index'].item() - physDF.loc[physDF_idx, 'Start Index'].item()
    pulse_stop = pulse_start + pulse_duration
    pulse_window = (pulse_start / 1000, pulse_stop / 1000)

    phys_ch_slice = phys_data._ch_to_index(phys_channel)

    ax[4].plot(HR_plot_time, np.squeeze(phys_data.array[phys_ch_slice, HR_plot_start:HR_plot_stop]))
    ax[4].set( ylabel = 'HR (bpm)', title='Cardiac')

    #Tonic Stim window
    ax[4].axvspan(pulse_window[0], pulse_window[1], color = 'b', alpha = 0.1, label='Tonic Stim')

    #HF window
    HF_start = 30 + metaDF.loc[ metaDF['TDT Tank'] == tank_ID, 'HF Delay (s)'].item()  #30 is from the 30 second pre-tonic stim plot window
    HF_stop = HF_start + HF_duration

    ax[4].axvspan(HF_start, HF_stop, color = 'r', alpha = 0.3, label='HF')
    ax[4].legend(loc='upper right')

    fig_title = metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['Type'].item() + ': '\
                + str(metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['HF Amplitude (mA)'].item()) + ' mA HF || '\
                + str(abs(stimDF.loc[param, 'pulse amplitude (μA)']) / 1000) + ' mA LF ' + 'Ch. ' + str(stimDF.loc[param, 'channel'])\
                + ' || ' + str(param)

    save_title = metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['Type'].item() + '_'\
               + str(metaDF.loc[ metaDF['TDT Tank'] == tank_ID]['HF Amplitude (mA)'].item()) + '_ma_HF_Ch'\
                + str(stimDF.loc[param, 'channel'])+'_StimAmp_'+ str(int(abs(stimDF.loc[param, 'pulse amplitude (μA)'])))+'_' + str(param)+'.png'
    print(save_title)
    fig.suptitle(fig_title)
    fig.tight_layout()

    if save_fig == True:
        fig.savefig(save_title, format='png')
        fig.clear()
        plt.clf()
        plt.close()
    else:
        plt.show()

def ADI_align(phys_data, ADI_filePATH, metaDF, stim):
    """ Physiology Data Import"""
    comMATRIX = sio.loadmat(ADI_filePATH)
    
    """Import comments into new more accessible DF and align them with sample #'s of the entire file"""
    comDF = pd.DataFrame()
    comDF[['Comment Text' ,'Sample #', 'Stim Train ID']] = ""
    comDF['Comment Matrix ID'] = comMATRIX['com'][:,4]
    
    for i,V in enumerate(comDF['Comment Matrix ID']): 
        comDF.loc[i, 'Comment Text'] = comMATRIX['comtext'][int(V-1)].strip()
    
    #Calculates the # samples per ADI block
    blockLENGTHS = comMATRIX['dataend'] - comMATRIX['datastart']
    blockIDX = []
    
    #Constructs a list of tuples containing the start and stop indices of each ADI block.
    #This is necessary to find the exact sample # of each comment due to how they are mapped by ADI i.e. the 'com' array gives the index of the comment within the block it was made -- not for the whole file
    for i,V in enumerate(blockLENGTHS[0,:]):
        if i == 0:
            blockIDX.append((0,V))
            off = V
        else:
            on = off + 1
            off = off + V
            blockIDX.append((on,off))
    
    #Iterate through comments matrix to find the index values of the comments and assigns them to the comment list
    for i,V in enumerate(comMATRIX['com']):
        blockNUM = int(V[1] - 1) #V[1] is the block that # that the comment was made in.  Subtract 1 to get it into python indexing (starts @ 0)    
        comDF.loc[i, 'Sample #'] = blockIDX[blockNUM][0] + V[2] #blockIDX[blockNUM][0] = Starting sample # of the block the comment is in.  V[2] is the sample # within that block that the comment was made.
    
    """ Code segment which looks for on/offs in the stim output from the TDT to ADI.  If the ADI data is noisy or messed up this may need tweaking.  Generally, there should be an equal number of start / stop indices assuming the system started with the stim trigger output on 0V."""
    phys_ch_slice = phys_data._ch_to_index('Stim Trigger')
    dataDF = pd.DataFrame(data = np.squeeze(phys_data.array[phys_ch_slice].compute()), columns=['Stim Trigger'])
    
    start_idx = dataDF.loc[(dataDF['Stim Trigger'] > 2.5) & (dataDF['Stim Trigger'].shift() <2.5) & (dataDF['Stim Trigger'].shift(10) < 0.5)].index
    print('Found ' + str(len(start_idx)) + ' start indices')
    stop_idx = dataDF.loc[(dataDF['Stim Trigger'] < 0.5) & (dataDF['Stim Trigger'].shift() > 0.5) & (dataDF['Stim Trigger'].shift(10) > 3)].index
    #TODO: make this an if statement check?
    #stop_idx = stop_idx[1:] #This line removes the first element of the stop indices that would occur if the system started with the stim trigger ouput at 3.3V
    print('Found ' + str(len(stop_idx)) + ' stop indices')
    
    """Creates a dataframe for mapping comments to stim pulse trains"""
    mapDF = pd.DataFrame(columns = ['Comment Text', 'Start Index', 'Stop Index', 'Stim Parameter', 'Max HR Change'])
    
    """Assigns start/stop indexes every pulse train (in order)"""
    mapDF['Start Index'] = start_idx
    mapDF['Stop Index'] = stop_idx
    mapDF['#Samples/Stim Train'] = mapDF['Stop Index'] - mapDF['Start Index']
    
    """ Lines up the comments with the pulse train by finding the closest pulse train start sample # to the comment's sample # """
    for i,V in enumerate(comDF['Sample #']):
       dist = (mapDF['Start Index'] - V).abs()
       mapDF.loc[dist.idxmin(), 'Comment Text'] = comDF['Comment Text'][i]
    
    """Up to this point everything has been done with just ADI data. The code below pulls in data from the stimulation
    object of pyeCAP to match up the stim parameters with the ADI data."""
    
    """Finds the pulse train # which lines up with the specific comment corresponding to each TDT tank."""
    comment_to_pulse_indicies = []
    for idx,com in enumerate(metaDF['ADI File Comment']):
        try:
            comment_to_pulse_indicies.append(mapDF.index[mapDF['Comment Text'] == com][0])
        except IndexError:
            traceback.print_exc()
            print('Could not find a comment match for TDT Tank: ' + metaDF.loc[idx, 'TDT Tank'])
            print('Erred Comment: ' + str(com))
            break
    #comment_to_pulse_indicies = [mapDF.index[mapDF['Comment Text'] == com][0] for com in metaDF['ADI File Comment']]
    
    """Maps the individual pulse trains to their specific stim index parameters"""
    stats_index_dict = {key:value for (key,value) in zip(metaDF['TDT Tank'].to_list(), comment_to_pulse_indicies)}
    grouped_parameters = stim.parameters.groupby(level=0).groups
    
    for tank_idx, tank in enumerate(metaDF['TDT Tank'].to_list()):
        for i in np.arange(grouped_parameters[tank_idx].size):
            mapDF.at[stats_index_dict[tank] + i, 'Stim Parameter'] = grouped_parameters[tank_idx][i]
    return mapDF

def get_parameters(type=None, HF_amp = None, LF_amp = None, stim_ch = None):
    paramLIST = []
    #Only Type is given
    if type is not None and HF_amp is None and LF_amp is None and stim_ch is None:
        tanks = metaDF.loc[(metaDF['Type'] == type)]['TDT Tank'].to_list()
        for tank in tanks:
            paramLIST.extend(tank_to_params_dict[tank].to_list())
    #Only HF_amp is given
    elif type is None and HF_amp is not None and LF_amp is None and stim_ch is None:
        tanks = metaDF.loc[ (metaDF['HF Amplitude (mA)'] == HF_amp)]['TDT Tank'].to_list()
        for tank in tanks:
            paramLIST.extend(tank_to_params_dict[tank].to_list())
    #Only Type and HF amp are given
    elif type is not None and HF_amp is not None and LF_amp is None and stim_ch is None:
        tanks = metaDF.loc[ (metaDF['Type'] == type) & (metaDF['HF Amplitude (mA)'] == HF_amp)]['TDT Tank'].item()
        paramLIST.extend(tank_to_params_dict[tanks])  
    else:
        tank = metaDF.loc[ (metaDF['Type'] == type) & (metaDF['HF Amplitude (mA)'] == HF_amp)]['TDT Tank'].item()
        paramDF = stim.parameters.loc[tank_to_params_dict[tank]]
        paramLIST = paramDF.loc[ (paramDF['pulse amplitude (μA)'] == LF_amp) & (paramDF['channel'] == stim_ch)].index.item()
    return paramLIST

def find_HF(param, ECAP_data, ECAP_channel, metaDF, tank_ID, plot = False):
    data = ECAP_data.array(param, ECAP_channel)[param]
    pulse_std = np.std(data, axis=2) ** 2
    
    HF_delay = metaDF.loc[ metaDF['TDT Tank'] == tank_ID, 'HF Delay (s)'].item()
    HF_duration = metaDF.loc[ metaDF['TDT Tank'] == tank_ID, 'HF Duration (s)'].item()
    
    pulses_per_sec = ECAP_data.stim.parameters.at[param, 'frequency (Hz)']
    baseline_pulse_stop = int((HF_delay - 1) * pulses_per_sec)
    baseline = np.mean(pulse_std[ 0 : baseline_pulse_stop])  #Mean st.dev during baseline period
    
    HF_window_start = int( (HF_delay + 2) * pulses_per_sec)
    HF_window_stop = int(( HF_delay + HF_duration - 2) * pulses_per_sec)
    HF = np.mean(pulse_std[ HF_window_start : HF_window_stop ])  #Mean st.dev during HF period
    
    HF_threshold = HF - ((HF - baseline)*0.5) #Mean of HF period - ((Mean of HF period - Mean of baseline) * 0.5) 
    #HF_threshold = 5 * baseline
    
    HF_indices = np.nonzero(pulse_std > HF_threshold)[0]
    HF_onset_pulse = HF_indices[0]
    HF_offset_pulse = HF_indices[-1]
    
    if plot:
        plt.plot(np.arange(data.shape[0]), pulse_std)
        #plt.axhline(baseline, color='c')
        #plt.axhline(HF, color='g')
        plt.axhline(HF_threshold, color='r')
        plt.show()
    
    return [HF_onset_pulse, HF_offset_pulse]

In [ ]:
"Import meta data"
"""Get HF meta data from excel file"""
excel_path = r'E:\HF_Block\20240904_HFBlock_Pig05\20240904_HFBlock_05_Parameters_modified.xlsx'
metaDF = pd.read_excel(excel_path, sheet_name = 'HF05_metaDF')

"""Remove rows that are not needed for current analysis and reset index of metaDF"""
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'HF Only'].index, inplace = True)  #Removed HF only tanks as they did not have any stim parameters
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Control'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'X'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Thresholding'].index, inplace = True)
metaDF.drop(metaDF.loc[ metaDF['Type'] == 'Contact ID'].index, inplace = True)
metaDF.reset_index(inplace = True)

"TDT Data Import - List"
tdt_path = r'E:\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Lab Path
#tdt_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05\HF_Block_Template-240904-083702' #Local Path 
tdt_file_list_all = os.listdir(tdt_path)
tdt_file_list_all.remove('desktop.ini')

ADI_path = r'E:\HF_Block\20240904_HFBlock_Pig05' #Lab Path
#ADI_path = r'G:\Data\HF_Block\20240904_HFBlock_Pig05' #Local path
ADI_file = r'\20240904_HFBlock_Pig05_stats_modified.mat'

raw_phys = pyeCAP.Phys(ADI_path + ADI_file, mult_data=False)
ws_stripped_names = [i.strip() for i in raw_phys.ch_names] #This line just removes the white spaces around the names imported
raw_phys = raw_phys.set_ch_names(ws_stripped_names)

"""Ephys data instantiation"""
tdt_path_list = [tdt_path + '\\' + fileNAME for fileNAME in metaDF['TDT Tank']]

#Data Streams
raw_ECAP = pyeCAP.Ephys(tdt_path_list, stores = 'ECAP')
raw_ECAP = raw_ECAP.remove_ch(['ECAP 6', 'ECAP 7', 'ECAP 8'])

raw_triCAP = pyeCAP.Ephys(tdt_path_list, stores = 'TriP')
raw_triCAP = raw_triCAP.remove_ch(['TriP 2', 'TriP 3', 'TriP 4'])

raw_EMG = pyeCAP.Ephys(tdt_path_list, stores='EMGG')
raw_EMG = raw_EMG.remove_ch(['EMGG 4'])

"""Changes the stim multi-index to utilize the Tank names from tdt_file_list instead [0,1,2,3...]"""
stim = pyeCAP.Stim(tdt_path_list)
stimDF = stim.parameters
tank_to_params_dict = { tank:params for (tank,params) in zip(metaDF['TDT Tank'].to_list(), stim.parameters.groupby(level=0).groups.values() ) }
params_to_tank_dict = { param:tank for (param,tank) in zip(stim.parameters.groupby(level=0).groups.keys(), metaDF['TDT Tank'].to_list()) }

#HF output from Keithley DAQ
HF_output = pyeCAP.Ephys(tdt_path_list, stores='HF_a')

"""Filtering - Standard"""
#raw_ECAP_f = raw_ECAP.filter_powerline()
raw_ECAP_f = raw_ECAP.filter_gaussian(Wn=5000, btype='lowpass')
raw_ECAP_f = raw_ECAP_f.filter_median(btype='highpass')

#raw_triCAP_f = raw_triCAP.filter_powerline()
raw_triCAP_f = raw_triCAP.filter_gaussian(Wn=5000, btype='lowpass')
raw_triCAP_f = raw_triCAP_f.filter_median(btype='highpass')

#raw_EMG_f = raw_EMG.filter_powerline()
raw_EMG_f = raw_EMG.filter_gaussian(Wn=5000, btype='lowpass')
raw_EMG_f = raw_EMG_f.filter_median(btype='highpass')

"""ECAP/EMG/TriCAP object instantiation"""
ECAPr = pyeCAP.ECAP(raw_ECAP, stim)
triCAPr = pyeCAP.ECAP(raw_triCAP, stim)
ECAPf = pyeCAP.ECAP(raw_ECAP_f, stim)
triCAPf = pyeCAP.ECAP(raw_triCAP_f, stim)
EMGr = pyeCAP.EMG(raw_EMG, stim)
EMGf = pyeCAP.EMG(raw_EMG_f, stim)

"Align stimulations with ADI data"
#physDF = ADI_align(phys_data, ADI_path + ADI_file, metaDF, stim=stim)
PHYSr = pyeCAP.ECAP(raw_phys,stim)

In [ ]:
#Onset/Offset analysis
#Get an RMS on a pulse by pulse basis
#Plot that RMS across periods of HF onset/offset
#Array returns a dictionary of data with the key = parameter tuple and value = data of pulses and channels
#Need to iterate through that somehow if multiple parameters get passed and return a new dictionary of RMS values
#RMS array should have one value per pulse

In [ ]:
"""Testing using ECAPr pulse RMS to find HF"""
files = metaDF['TDT Tank'].loc[ metaDF['Type'] == 'EMG DRC'].values.tolist()
params = get_parameters(type='EMG DRC')
plot_params = get_parameters(type='EMG DRC', HF_amp=0.5)

In [ ]:
for param in plot_params:
    HF_onset, HF_offset = find_HF(param, ECAPr, ECAP_channel=0, metaDF=metaDF, tank_ID=params_to_tank_dict[param[0]], plot=True)
    print(param)
    print(HF_onset, HF_offset)

In [ ]:
param = (3,0)
bin = [335,336]
ECAPr.plot_binned_traces(parameter = param, channel=0, bin=bin, y_lim = (-100,100))

data = ECAPr.array(param, 0)[param]
tank_ID = params_to_tank_dict[param[0]]

pulse_std = np.std(data, axis=2)  #Array of a single standard deviation values for each pulse

HF_delay = metaDF.loc[metaDF['TDT Tank'] == tank_ID, 'HF Delay (s)'].item()
HF_duration = metaDF.loc[metaDF['TDT Tank'] == tank_ID, 'HF Duration (s)'].item()

pulses_per_sec = ECAPr.stim.parameters.at[param, 'frequency (Hz)']
baseline_pulse_stop = int((HF_delay - 1) * pulses_per_sec)
baseline = np.mean(pulse_std[0: baseline_pulse_stop])

HF_window_start = int((HF_delay + 2) * pulses_per_sec)
HF_window_stop = int((HF_delay + HF_duration - 2) * pulses_per_sec)
HF = np.mean(pulse_std[HF_window_start: HF_window_stop])
#HF_threshold = HF - ((HF - baseline) * 0.33)
HF_threshold = 5 * baseline

HF_indices = np.nonzero(pulse_std > HF_threshold)[0]
HF_onset_pulse = HF_indices[0]
HF_offset_pulse = HF_indices[-1]

#Iterate through each pulse and find the first pulse where np.max greater than some value?

#HF_stop_threshold = HF - ((HF - baseline) * 0.33)
#HF_stop_indices = np.nonzero(pulse_std < HF_stop_threshold)[0]
#First value in HF_stop_indices that is after the HF_onset_pulse value
#HF_stop_idx2 = np.nonzero(HF_stop_indices > HF_onset_pulse)[0]
#print(HF_stop_idx2)
#HF_offset_pulse = HF_stop_indices[HF_stop_idx2[0]]

print(HF_onset_pulse,HF_offset_pulse)
plt.plot(np.arange(data.shape[0]), pulse_std)
#plt.axhline(baseline, color='c')
#plt.axhline(HF, color='g')
plt.axhline(HF_threshold, color='r')
plt.xlim(bin[0], bin[1])
plt.show()

In [ ]:
testdata = ECAPr.array(param, 0)[param]
testdata.shape

In [ ]:
ser = pd.Series(np.squeeze(testdata[137]))
plt.plot(ser)
plt.show()

In [ ]:
ser = pd.Series(np.squeeze(testdata[137]))
rolling_output = ser.rolling(250).std() * 1e6

plt.plot(rolling_output)
plt.show()

In [ ]:
ser = pd.Series(np.squeeze(testdata[137]))
rolling_output = ser.rolling(250).std() * 1e6

plt.plot(rolling_output)
plt.axhline(HF_threshold*1e6, color='r')
plt.show()

In [ ]:
250 * 1 / ECAPr.sample_rate * 1e3


In [ ]:
stimDF

In [ ]:
files = metaDF['TDT Tank'].loc[ metaDF['Type'] == 'EMG DRC'].values.tolist()
pulse_RMS = {}
for file in files:
    print(file)
    pulse_RMS.update( EMGf.pulse_RMS(parameters = tank_to_params_dict[file].tolist()))

In [ ]:
params = get_parameters(type = 'EMG DRC', HF_amp=0.5)
chan = 1
for param in params:
    fig,ax = plt.subplots(figsize=(10,5))
    ax.plot(np.arange(pulse_RMS[param].shape[0]), pulse_RMS[param][:,chan])  
    ax.set(title=param)
    plt.show()

In [ ]:
onset,offset = find_HF((0,8), ECAPr, ECAP_channel=0, metaDF = metaDF, tank_ID='HF05-240904-120445', plot=True)
print(onset)
print(offset)

In [ ]:
param = (0,8)
EMGf.plot_binned_traces(parameter = param, channel=1, bin=[130,140], y_lim = (-50,50))

In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
ax.plot(np.arange(pulse_RMS[param].shape[0]), pulse_RMS[param][:,chan])  
ax.set(title=param)#, xlim=(0,100))
plt.show()

In [ ]:
param = (0,0)
ECAPr.plot_binned_traces(parameter = param, channel=0, bin=[130,133], y_lim = (-50,50))